In [24]:
from sqlalchemy import create_engine
import pandas as pd

In [57]:
# create path to data source
crowdcsv = 'rawdata/data.csv'
membercsv = 'rawdata/multiTimeline.csv'

In [64]:
# use pandas read_csv() function to read in data
c = pd.read_csv(crowdcsv)
m = pd.read_csv(membercsv, skiprows=1)
# extract only the columns we want to use
gym = c[['timestamp','number_people','date','temperature']].copy()

# rename columns and create an id column (didn't work with timestamp so i created an id column)
crowd = gym.rename(columns={'timestamp':'id',
                           'number_people':'members_workingout',
                           'date':'date',
                           'temperature':'temp'})
membership = m.rename(columns={'Week':'date',
                               'gym membership: (United States)':'membership_count'})
# converting date column to date without time
crowd['date'] = pd.to_datetime(crowd['date'], utc=True).dt.date
# dropping duplicates in the dataset
crowd.drop_duplicates(subset=['date'],inplace=True)
membership.drop_duplicates(inplace=True)
# reset index after dropping dupes
crowddf = crowd.reset_index()
# drop incorrect index column
crowddf.drop('index',axis=1,inplace=True)
crowddf.head()

,id,members_workingout,date,temp
0,61211,37,2015-08-15,71.76
1,61215,26,2015-08-16,77.32
2,61213,31,2015-08-17,78.79
3,61210,47,2015-08-18,67.04
4,61213,45,2015-08-19,65.96


In [65]:
membership.head()

,date,membership_count
0,2015-08-16,55
1,2015-08-23,66
2,2015-08-30,52
3,2015-09-06,47
4,2015-09-13,43


In [63]:
# see what type of data each column is for pgAdmin
crowddf.dtypes
membership.dtypes

date                object
membership_count     int64
dtype: object

In [60]:
# create connection to pgAdmin
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5433
database_name = 'project2'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [61]:
# check to see your table in pgAdmin is linked
engine.table_names()

/Users/benraseman/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  


['crowd', 'members']

In [62]:
# load data into pgAdmin
crowddf.to_sql(name='crowd',con=engine,if_exists='append',index=False)
membership.to_sql(name='members',con=engine,if_exists='append',index=False)